In [ ]:
https://towardsdatascience.com/multi-output-model-with-tensorflow-keras-functional-api-875dd89aa7c6

In [1]:
from sklearn.datasets import load_boston 
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
boston = load_boston()

/home/matias/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing data

In [3]:
data = pd.DataFrame(boston.data)
data.columns = boston.feature_names
data['PRICE'] = boston.target
data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [4]:
train,test = train_test_split(data, test_size=0.2, random_state = 1)
train,val = train_test_split(train, test_size=0.2, random_state = 1)

In [5]:
train_stats = train.describe()
train_stats.pop('PRICE')
train_stats.pop('PTRATIO')

count    323.000000
mean      18.483282
std        2.162864
min       12.600000
25%       17.350000
50%       19.100000
75%       20.200000
max       22.000000
Name: PTRATIO, dtype: float64

In [6]:
train_stats = train_stats.transpose()

In [7]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

def format_output(data):
    y1 = data.pop('PRICE')
    y1 = np.array(y1)
    y2 = data.pop('PTRATIO')
    y2 = np.array(y2)
    return y1, y2

In [8]:
train_Y = format_output(train)
test_Y = format_output(test)
val_Y = format_output(val)

In [9]:
norm_train_X = np.array(norm(train))
norm_test_X = np.array(norm(test))
norm_val_X = np.array(norm(val))

In [12]:
def build_model():
    # Define model layers.
    input_layer = tf.keras.layers.Input(shape=(len(train.columns),))
    
    first_dense = tf.keras.layers.Dense(units='128', activation='relu')(input_layer)
    # Y1 output will be fed from the first dense
    y1_output = tf.keras.layers.Dense(units='1', name='price_output')(first_dense)
    
    second_dense = tf.keras.layers.Dense(units='128',activation='relu')(first_dense)
    # Y2 output will be fed from the second dense
    y2_output = tf.keras.layers.Dense(units='1',name='ptratio_output')(second_dense)    # Define the model with the input layer 
    # and a list of output layers
    model = tf.keras.Model(inputs=input_layer,outputs=[y1_output, y2_output])
    return model

In [ ]:
model = build_model()# Specify the optimizer, and compile the model with loss functions for both outputs

optimizer = tf.keras.optimizers.SGD(lr=0.001)

model.compile(optimizer=optimizer,
              loss={'price_output': 'mse', 'ptratio_output': 'mse'},
            
              metrics={'price_output':tf.keras.metrics.RootMeanSquaredError(),
                      'ptratio_output':tf.keras.metrics.RootMeanSquaredError()})

In [ ]:
# Train the model for 100 epochs
history = model.fit(norm_train_X, train_Y,
   epochs=100, batch_size=10, validation_data=(norm_test_X, test_Y))

In [ ]:
loss,Y1_loss,Y2_loss,Y1_rmse,Y2_rmse=model.evaluate(x=norm_val_X, y=val_Y)print()
print(f'loss: {loss}')
print(f'price_loss: {Y1_loss}')
print(f'ptratio_loss: {Y2_loss}')
print(f'price_rmse: {Y1_rmse}')
print(f'ptratio_rmse: {Y2_rmse}')

In [ ]:
# Run predict
Y_pred = model.predict(norm_test_X)
price_pred = Y_pred[0]
ptratio_pred = Y_pred[1]plot_diff(test_Y[0], Y_pred[0], title='PRICE')
plot_diff(test_Y[1], Y_pred[1], title='PTRATIO')

In [ ]:
# Save model
model.save('./model_boston/', save_format='tf')

In [ ]:
# Restore model
loaded_model = tf.keras.models.load_model('./model_boston/')

In [ ]:
# Run predict with restored model
predictions = loaded_model.predict(norm_test_X)
price_pred = predictions[0]
ptratio_pred = predictions[1]